In [ ]:
# Install pyspark
!pip install pyspark

# Import SparkSession
from pyspark.sql import SparkSession

# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()

# Check Spark Session Information
spark

# Import a Spark function from library
from pyspark.sql.functions import col

!pip install findspark


You are given housing prices for a given market (WestRoxbury.csv Download WestRoxbury.csv)
Create a Jupyter notebook that analyzes this data using PySpark. Load the data into a pyspark data frame and conduct any necessary datatyping/casting. In the notebook, you must answer the following questions using code. 
1) Identify the top 10 most expensive homes.
2) Does remodeling a home indicate higher prices (simply compare the average between recently remodeled homes versus those that were not).
3) Create a linear regression model that predicts home prices using LOT SQFT, YR Built, Gross Area, Living Area, and REMODEL variables. 
4) Using your predictive model, determine the price of a home with the following:
LOT SQRT: 7500
YR Built: 1990
Gross Area: 2100
Living Area: 1900
Remodel: Recent


In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession;

spark = SparkSession.builder.config("spark.driver.host","localhost").master("local[4]").appName("ISM6562 Spark Assignment App").getOrCreate();

# Let's get the SparkContext object. It's the entry point to the Spark API. It's created when you create a sparksession
sc = spark.sparkContext  

# note: If you have multiple spark sessions running (like from a previous notebook you've run), 
# this spark session webUI will be on a different port than the default (4040). One way to 
# identify this part is with the following line. If there was only one spark session running, 
# this will be 4040. If it's higher, it means there are still other spark sesssions still running.
spark_session_port = spark.sparkContext.uiWebUrl.split(":")[-1]
print("Spark Session WebUI Port: " + spark_session_port)

In [ ]:
# this will set the log level to ERROR. This will hide the INFO or WARNING messages that are printed out by default. If you want to see them, set this to INFO or WARN.
sc.setLogLevel("ERROR") 

In [ ]:
spark

## Loading our data into spark dataframe. 

In [ ]:
# Load CSV file
df_spark = spark.read.csv("sample_data/WestRoxbury.csv", header=True, inferSchema=True)
df_spark.show()

## Rename Column 

In [ ]:
df_renamed = df_spark.withColumnRenamed("TOTAL VALUE ","total_value").withColumnRenamed("TAX","tax").withColumnRenamed("LOT SQFT ","lot_sqft").withColumnRenamed("YR BUILT","yr_built").withColumnRenamed("GROSS AREA ","gross_area").withColumnRenamed("LIVING AREA","living_area").withColumnRenamed("FLOORS ","floors").withColumnRenamed("ROOMS","rooms").withColumnRenamed("BEDROOMS ","bedrooms").withColumnRenamed("FULL BATH","full_bath").withColumnRenamed("HALF BATH","half_bath").withColumnRenamed("KITCHEN","kitchen").withColumnRenamed("FIREPLACE","fireplace").withColumnRenamed("REMODEL","remodel")

df_renamed.show()

# Data Exploration and Transformations 

In [ ]:
df_renamed.printSchema()

# Finding missing values

In [ ]:
from pyspark.sql.functions import col

# Find columns with missing values
columns_with_missing_values = [column for column in df_renamed.columns if df_renamed.filter(col(column).isNull()).count() > 0]

# Print columns with missing values
print("Columns with missing values:")
for column in columns_with_missing_values:
    print(column)


# Imputing missing values

In [ ]:
#from pyspark.sql.functions import col

# Group by the column and apply the count() function
#count_df = df_renamed.groupBy("TypeofContact").count()

# Show the resulting counts
#count_df.show()

In [ ]:
#df_spark = df_renamed.fillna("Self Enquiry", subset=["TypeofContact"])

Imputing with missing value with median value for numeric values

In [ ]:
# #from pyspark.sql.functions import col
# from pyspark.sql.functions import percentile_approx
# from pyspark.sql.functions import when

# # Iterate over columns with missing values
# for column in columns_with_missing_values:
#     # Calculate median of the column
#     median_value = df_spark.select(column).agg(percentile_approx(column, 0.5)).collect()[0][0]
#     if median_value is not None:
#     # Round median_value to nearest integer
#         median_value_rounded = int(round(median_value))
#     else:
#         median_value_rounded = 0
    
#     # Impute missing values with median value
#     df_spark = df_spark.withColumn(column, when(col(column).isNull(), median_value_rounded).otherwise(col(column)))

In [ ]:
# # verifying if all missing values were imputed
# columns_with_missing_values = [column for column in df_spark.columns if df_spark.filter(col(column).isNull()).count() > 0]

# # Print columns with missing values
# print("Columns with missing values:")
# for column in columns_with_missing_values:
#     print(column)

In [ ]:
# # Save imputed data with original header

# df_spark.write.option("header", True).csv("/content/Big Data Output/processed_travel_withheader.csv")

In [ ]:
# df_spark.show(20)

# Storing the data into a persistent table and creating a temp view of the data


In [ ]:
# Create a database
spark.sql("CREATE DATABASE IF NOT EXISTS HOUSING")

# Use the database
spark.sql("USE HOUSING")

In [ ]:
#spark.sql("DROP DATABASE HOUSING")

In [ ]:
# To drop the Table
#spark.sql("DROP TABLE house_sale")

In [ ]:
# Store the data into a persistent table in the Travel db
df_renamed.write.saveAsTable("house_sales")

In [ ]:
# Create a temporary view of the data
df_renamed.createOrReplaceTempView("housing_view")

In [ ]:
# Verify if the table exists in the created database
check = spark.sql("SHOW TABLES")
if check.filter(check.tableName == "housing_view").count() > 0:
    print("Table exists in the created database.")

# Now let us find some insights by using the aggregation.

<p style="color:brown;">1) Identify the top 10 most expensive homes. </p>


In [ ]:
top_exp_homes = spark.sql("SELECT * FROM housing_view order by total_value desc limit 10").show()


<p style="color:brown;">2) Does remodeling a home indicate higher prices (simply compare the average between recently remodeled homes versus those that were not).</p>


In [ ]:
total_result = spark.sql("""
    select remodel,avg(total_value) from housing_view group by remodel
""").show()

3) Create a linear regression model that predicts home prices using LOT SQFT, YR Built, Gross Area, Living Area, and REMODEL variables. 

Let's select a the subset of columns we are interested in for our analysis.

In [ ]:
df_data =df_renamed.select([
    'lot_sqft',
    'yr_built',
    'gross_area',
    'living_area',
    'remodel',
    'total_value'
    ]
)

In [ ]:
df_data.count()

In [ ]:
#Remove any rows from the DataFrame that contain missing values
df_data = df_data.dropna()


In [ ]:
df_data.count()

In [ ]:
df_data.show()

#Model Training

We earlier created the incident table with 'inferSchema' set to true. This means that the schema of the table is inferred from the data. The knowledge column was properly inferred as a boolean, however, in the pipeline below with use StringIndexer on this column - and therefore, need to convert this column data type to a string.

In [ ]:
df_data.printSchema()

CONVERTING BOOLEAN DATA TO NUMERICAL


In [ ]:
#from pyspark.sql.types import IntegerType,BooleanType,DateType, StringType

#df_closed_incidents = df_data.withColumn("knowledge",df_data.knowledge.cast(StringType()))

In [ ]:
#df_closed_incidents.printSchema()

Now that we have our data ready, let's do a train test split (70/30).

In [ ]:
train_data,test_data=df_data.randomSplit([0.7,0.3])

CONVERTING STRING TO NUMERICAL

In [ ]:
from pyspark.ml.feature import StringIndexer
# Use StringIndexer to convert the categorical columns to hold numerical data
 
remodel_indexer = StringIndexer(inputCol='remodel',outputCol='remodel_indexer',handleInvalid='keep')


In [ ]:
from pyspark.ml.feature import VectorAssembler
# Vector assembler is used to create a vector of input features
 
assembler = VectorAssembler(
    inputCols=[
        "remodel_indexer",
        "lot_sqft",
        "yr_built",
        "gross_area",
        "living_area"   
    ],
    outputCol="features"
)

In [ ]:
from pyspark.ml import Pipeline

# Pipeline is used to pass the data through indexer and assembler simultaneously. Also, it helps to pre-rocess the test data
# in the same way as that of the train data
# https://spark.apache.org/docs/latest/ml-pipeline.html
 
pipe = Pipeline(stages=[
    remodel_indexer,
    assembler
    ]
)

In [ ]:
fitted_pipe=pipe.fit(train_data)

In [ ]:
train_data=fitted_pipe.transform(train_data)
train_data.show()

In [ ]:
test_data=fitted_pipe.transform(test_data)
test_data.show()

In [ ]:
# # For those interested in utilizing the ML/AI power of Tensorflow with Spark....
# # https://github.com/tensorflow/ecosystem/tree/master/spark/spark-tensorflow-distributor

# # In this course, we'll use the SparkML (admitedely, it's not as powerful as Tensorflow, but 
# # it's easy to use and demonstrate ML on a Spark Cluster)

# from pyspark.ml.regression import LinearRegression

lr_model = LinearRegression(labelCol='total_value')
fit_model = lr.fit(train_data.select(['features','total_value']))


In [ ]:
results = fit_model.transform(test_data)
results.show()

In [ ]:
results.select(['location_index','prediction']).show()

Evaluate the peformance of the Linear Regression Model

In [ ]:
test_results = fit_model.evaluate(test_data)

In [ ]:
test_results.residuals.show()

In [ ]:
print(f"{'RMSE:':7s} {test_results.rootMeanSquaredError:>7.3f}")
print(f"{'Ex Var:':7s} {test_results.explainedVariance:>7.3f}")
print(f"{'MAE:':7s} {test_results.meanAbsoluteError:>7.3f}")
print(f"{'MSE:':7s} {test_results.meanSquaredError:>7.3f}")
print(f"{'RMSE:':7s} {test_results.rootMeanSquaredError:>7.3f}")
print(f"{'R2:':7s} {test_results.r2:>7.3f}")

4) Using your predictive model, determine the price of a home with the following:
LOT SQRT: 7500
YR Built: 1990
Gross Area: 2100
Living Area: 1900
Remodel: Recent


In [ ]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder

from pyspark.ml.linalg import Vectors

# Create a test dataframe with the given features

new_home_data = [(7500, 1990, 2100, 1900, 1)]

test_df = spark.createDataFrame(new_home_data, ['lot_sqft', 'yr_built', 'gross_area', 'living_area', 'remodel_by_index'])

In [ ]:
home_df = assembler.transform(test_df)

In [ ]:
new_home_features =Vectors.dense(home_df.select('features').collect()[0][0])
predicted_price = fit_model.predict(new_home_features)


In [ ]:
print(f'price is  {predicted_price:.2f} dollars ' )